In [ ]:
import pandas as pd
from twython import Twython
from nltk.tokenize import word_tokenize
from sklearn.cross_validation import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics


import datetime
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

import nltk.data
tokens = nltk.data.load('nltk:tokenizers/punkt/english.pickle')

In [ ]:
tweets = []
sentiment=[]
tweetId=[]



emoticons_str =     r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""

regex_str = [  # HTML tags
               # @-mentions 
               # hash-tags
               # URLs
               # numbers
               # words with - and '
               # other words
               # anything else
    emoticons_str,
    r'<[^>]+>',
    r'(?:@[\w_]+)',
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)",
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+',
    r'(?:(?:\d+,?)+(?:\.?\d+)?)',
    r"(?:[a-z][a-z'\-_]+[a-z])",
    r'(?:[\w_]+)',
    r'(?:\S)',
    ]

tokens_re = re.compile(r'(' + '|'.join(regex_str) + ')', re.VERBOSE
                          | re.IGNORECASE)
emoticon_re = re.compile(r'^' + emoticons_str + '$', re.VERBOSE | re.IGNORECASE)

In [ ]:
data=pd.read_csv('travelBan.csv')
dataTravelBan=pd.DataFrame(data)

data=pd.read_csv('parisAttacks.csv')
dataParis=pd.DataFrame(data)

data=pd.read_csv('syrianBombing.csv')
dataSyria=pd.DataFrame(data)

In [ ]:
def preprocess(s):
    s = s.lower()
    
    tokens = tokenize(s)
    stopset = set(stopwords.words('english'))
    #tokens = [item.lower() for item in tokens]
    tokens = [w for w in tokens if not w in stopset]
    tokens = [i for i in tokens if i not in string.punctuation]

    prefixes = ('https','@', ':','rt')
    new_tokens = []
    for token in tokens:
        if not token.startswith(prefixes) and len(token) != 1:
            if token not in new_tokens:
                new_tokens.append(token)
    lmtzr = WordNetLemmatizer()
    new_tokens = [lmtzr.lemmatize(token) for token in new_tokens]

    return new_tokens

def tokenize(s):
    return tokens_re.findall(s)

stemmer = PorterStemmer()

def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

In [ ]:
def saveDatabase(df):
    tweetsPost={}
    
    for i,row in d.iterrows():
        tweetTokenized=preprocess(row['Text'])
        if tweetTokenized not in tweets:
            tweets.append(tweetTokenized)
            sentiment.append(row['Sentiments'])
            location.append(row['Location'])

In [ ]:
saveDatabase(dataParis)
saveDatabase(dataSyrian)
saveDatabase(dataTravelBan)

In [ ]:
dataTrain=pd.DataFrame({'Tweets':tweets,'Sentiment':sentiment,'Location':location})
X = f.Tweets
y = f.Sentiment

In [ ]:
vect = CountVectorizer()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

train_transformed = []
test_transformed = []
list_y_train=y_train.tolist()
list_y_test=y_test.tolist()

In [ ]:
for i in X_train:
    train_transformed.append(' '.join(i))


X_train_transformed=vect.fit_transform(train_transformed)



for i in X_test:
    test_transformed.append(' '.join(i))

X_test_transformed = vect.transform(test_transformed)

In [ ]:
nb = MultinomialNB()
nb.fit(X_train_transformed, new_y_train)
y_pred_class = nb.predict(X_test_transformed)

In [ ]:
metrics.confusion_matrix(new_y_test, y_pred_class)
metrics.accuracy_score(new_y_test, y_pred_class)

In [ ]:
X_train_tokens = vect.get_feature_names()
token_count = nb.feature_count_


pos_token_count = nb.feature_count_[1, :]
neg_token_count = nb.feature_count_[2, :]
neu_token_count=nb.feature_count_[0, :]
tokens = pd.DataFrame({'token':X_train_tokens, 'neu':neutral_token_count, 'pos':pos_token_count,'neg':neg_token_count}).set_index('token')
